In [ ]:
# If you just want the basic functionality
pip install napari tifffile tqdm

# For better performance with large images, also install these:
pip install dask scikit-image

# Install napari with all plugins
pip install "napari[all]"

# Install additional packages for TIF handling
pip install tifffile scikit-image tqdm

In [ ]:
import os
import numpy as np
import napari
import tifffile
from tqdm import tqdm

def view_tif_stack_3d(path, scale=None, contrast_limits=None, colormap='gray'):
    """
    View a stack of TIF images as a 3D volume using Napari.
    
    Parameters:
    -----------
    path : str
        Path to the directory containing TIF files or path to a single multi-page TIF file
    scale : tuple, optional
        Scale factors for each dimension (z, y, x)
    contrast_limits : tuple, optional
        (min, max) values for contrast adjustment
    colormap : str, optional
        Colormap to use for visualization (default: 'gray')
    """
    # Check if the path is a directory or a file
    if os.path.isdir(path):
        # Get list of all TIF files in the directory
        tif_files = sorted([os.path.join(path, f) for f in os.listdir(path) 
                            if f.lower().endswith(('.tif', '.tiff'))])
        
        if not tif_files:
            print(f"No TIF files found in the directory: {path}")
            return
        
        print(f"Loading {len(tif_files)} TIF slices...")
        
        # Option 1: Load all slices into memory (may be memory-intensive for large stacks)
        # volume = np.stack([tifffile.imread(f) for f in tqdm(tif_files)], axis=0)
        
        # Option 2: Create a memory-mapped array if the files have consistent dimensions
        # Check dimensions of first image
        sample = tifffile.imread(tif_files[0])
        shape = (len(tif_files),) + sample.shape
        dtype = sample.dtype
        print(f"Creating volume with shape {shape} and dtype {dtype}")
        
        # Create a memory-mapped array
        volume = np.zeros(shape, dtype=dtype)
        
        # Load each slice
        for i, file in enumerate(tqdm(tif_files)):
            volume[i] = tifffile.imread(file)
            
    else:  # Assume it's a single multi-page TIF file
        if not os.path.exists(path):
            print(f"File not found: {path}")
            return
            
        print(f"Loading multi-page TIF file: {path}")
        # For multi-page TIF files, use tifffile's memory-mapping capabilities
        volume = tifffile.imread(path)
    
    print(f"Volume shape: {volume.shape}, dtype: {volume.dtype}")
    
    # Automatically determine contrast limits if not provided
    if contrast_limits is None:
        # Use percentiles to avoid outliers affecting the contrast
        p_low, p_high = np.percentile(volume, (0.5, 99.5))
        contrast_limits = (p_low, p_high)
        print(f"Auto contrast limits: {contrast_limits}")
    
    # If scale not provided, use isotropic scaling
    if scale is None:
        scale = (1.0, 1.0, 1.0)  # (z, y, x)
    
    # Start napari viewer
    viewer = napari.Viewer()
    
    # Add the volume layer
    viewer.add_image(
        volume,
        scale=scale,
        contrast_limits=contrast_limits,
        colormap=colormap,
        rendering='mip',  # 'mip' for maximum intensity projection, alternatives: 'attenuated_mip', 'iso', 'minip'
        name='TIF Stack'
    )
    
    # Add slice layers for orthogonal views
    viewer.add_image(
        volume,
        scale=scale,
        contrast_limits=contrast_limits,
        colormap=colormap,
        visible=False,
        name='XY Slice'
    )
    
    print("Visualization ready - use the slider to navigate through slices")
    print("Press 3 to toggle 3D view mode")
    print("Use CTRL+scroll to zoom in/out")
    
    # Make the viewer take up the available space
    viewer.window.resize(1024, 768)
    
    return viewer


# Example usage:
path = r"C:\Users\fedhila\Downloads\tif_slices-20250505T114141Z-001\tif_slices"

# Optional: Define scale for anisotropic data (z, y, x)
# If your X and Y resolution is 0.5μm and Z spacing is 2μm:
# scale = (2.0, 0.5, 0.5)

# Run the viewer
viewer = view_tif_stack_3d(path)
napari.run()